# Dfsu - Speed and direction

In [1]:
import numpy as np
from mikeio import Dfsu

filename = "../tests/testdata/HD2D.dfsu"
dfs = Dfsu(filename)
dfs

Dfsu2D
Number of elements: 884
Number of nodes: 529
Projection: UTM-29
Items:
  0:  Surface elevation <Surface Elevation> (meter)
  1:  U velocity <u velocity component> (meter per sec)
  2:  V velocity <v velocity component> (meter per sec)
  3:  Current speed <Current Speed> (meter per sec)
Number of items: 4
Time: 9 steps with dt=9000.0s
      1985-08-06 07:00:00 -- 1985-08-07 03:00:00

This file is missing current direction :-(

Lets'fix that!

In [2]:
ds = dfs.read()
ds

<mikeio.DataSet>
Dimensions: (9, 884)
Time: 1985-08-06 07:00:00 - 1985-08-07 03:00:00
Items:
  0:  Surface elevation <Surface Elevation> (meter)
  1:  U velocity <u velocity component> (meter per sec)
  2:  V velocity <v velocity component> (meter per sec)
  3:  Current speed <Current Speed> (meter per sec)

## Calculate speed & direction

In [3]:
u = ds["U velocity"]
v = ds["V velocity"]

In [4]:
direction = np.mod(90 -np.rad2deg(np.arctan2(v,u)),360)
speed = np.sqrt(u**2+v**2)

## Write new file

In [5]:
from mikeio.eum import ItemInfo, EUMType
from mikeio.dutil import Dataset
data = [speed, direction]

items = [ItemInfo(EUMType.Current_Speed),
         ItemInfo(EUMType.Current_Direction)]

newds = Dataset(data,ds.time,items)
newds

<mikeio.DataSet>
Dimensions: (9, 884)
Time: 1985-08-06 07:00:00 - 1985-08-07 03:00:00
Items:
  0:  Current Speed <Current Speed> (meter per sec)
  1:  Current Direction <Current Direction> (radian)

In [6]:
dfs.write("speed_direction.dfsu", newds)

## Clean up

In [7]:
import os

os.remove("speed_direction.dfsu")